# How To Use Vertex Text-to-Image Generative AI To Perform Mask Based Outpainting

This notebook outlines how to interact with Vertex AI's Text-to-Image GenAI models to create dynamic content using Outpainting. With Imagen on Vertex AI, developers can utilize next-generation AI products to transform their imagination into high quality visual assets, in seconds. With just some simple lines of Python, we can merge these assets with existing content to provide a new level of visual website content.

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
PROJECT_ID = "rkiles-demo-host-vpc"

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- google.auth - Provides authentication access to the Google API's, such as imagegeneration:predict
- PIL - An easy to use Python image library to help build the background for our banner and perform image layering
- io - Core python libray used to work with I/O. We will use this to help convert strings to byte objects for PIL
- base64 - Imagen API requests return generated or edited images as base64-encoded strings. This module will help us decode this data to an image file
- requests - This module will allow us to interact directly with Imagen over the REST API. 
- json - Python module used to interact with JSON data. Imagen returns results in json format.

In [ ]:
import google.auth.transport.requests
import google.auth
from PIL import Image
from io import BytesIO
import base64
import requests
import json

## Authenticate to the Vertex AI API

Our Vertex Workbench instance is configured to use a specified service account that has IAM access to the text-to-vision API. The following two secitons will allow us to generate the access token we will pass as an authorization bearer token later in the header.

In [ ]:
credentials, project_id = google.auth.default()
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)

In [ ]:
access_token = credentials.token

## Prepare the HTTP POST request to the REST API

Define the header fields, including the access token we created in the last step

In [ ]:
headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'application/json; charset=utf-8'
    }

You can uncomment the following line for troubleshooting if you want to see how the header will be passed to the API.

In [ ]:
#print(headers)

Next we will specifiy the URL for the Imagen REST API. You should have already specified the correct project ID in the very first step of this notebook.

In [ ]:
url = f'https://us-central1-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/us-central1/publishers/google/models/imagegeneration@006:predict'

To use Imagen on Vertex AI you must provide a text description of what you want to generate or edit. These descriptions are called prompts, and these prompts are the primary way you communicate with Generative AI. Here, we are specifiying what we want the model to create using a prompt. Play around with this content and see what kind of images you can create. More information can be found here https://cloud.google.com/vertex-ai/docs/generative-ai/image/img-gen-prompt-guide

In [ ]:
imagen_prompt = "Photo of a beautiful house with a large front lawn with leaves on the ground"

Specify the image and mask you want to use for outpainting. For this example, start with the mower_zoom-in.jpg and mask_mower_zoom-in.jpg files. Once you view the result, try it again with the zoom-out files to see how outpainting uses the mask and input images to determine the overall aspect of the final image.

In [ ]:
with open("mower_zoom-in.jpg", "rb") as f:
    encoded_base_image = base64.b64encode(f.read())
B64_BASE_IMAGE = encoded_base_image.decode('utf-8')

with open("mask_mower_zoom-in.jpg", "rb") as f2:
    encoded_mask_image = base64.b64encode(f2.read())
B64_MASK_IMAGE = encoded_mask_image.decode('utf-8')


In [ ]:
#print(B64_BASE_IMAGE)

The REST API expects a json payload containing the data that will be used to generate the image. We are only passing a few of the available request options below, but you can find more information about the REST API and additional featuers here https://cloud.google.com/vertex-ai/docs/generative-ai/image/edit-images

In this example, we are providing the prompt information from the last step and specifiying two image samples to be returned. You can also use 

In [ ]:
request_body = '{"instances": [{"prompt": "' + imagen_prompt + '","image": {"bytesBase64Encoded": "' + B64_BASE_IMAGE + '"},"mask": {"image": {"bytesBase64Encoded": "' + B64_MASK_IMAGE + '"}}}],"parameters": {"sampleCount": 2, "mode":"backgroundEditing"}}'

You can alternatively try Product Image Editing by uncommenting the line below. More information about Product image editing can be found here https://cloud.google.com/vertex-ai/docs/generative-ai/image/edit-images#gps-product-background

In [ ]:
#request_body = '{"instances": [{"prompt": "' + imagen_prompt + '","image": {"bytesBase64Encoded": "' + B64_BASE_IMAGE + '"}}],"parameters": {"IsProductImage": "True"}}'

Lastly, we will post the request to the Imagen REST API and wait for the requested image to be generated and returned

In [ ]:
r = requests.post(url, data=request_body, headers=headers)

You can optionally uncomment the following to view the returned status code for verification or troubleshooting

In [ ]:
print(r.status_code)
#print(r.content)

## Process the returned request and decode the image

The Imagen API returns images in a base64 encoded JSON string. We will start by defining our data and then decoding the image.

In [ ]:
img_data = r.json()

You can optionally uncomment this line to see the full JSON payload returned by the API.

In [ ]:
#print(img_data["predictions"][0]["bytesBase64Encoded"])

We will be using just the first returned image sample in this example. The API can return up to 4 sample images, so you could insert a simple for loop here to iterate through multiple images.

In [ ]:
encoded_image = img_data["predictions"][1]["bytesBase64Encoded"]

This will decode the base64 string to a usable image

In [ ]:
decoded_image = base64.b64decode(encoded_image)

Optionally uncomment this line to save the image to disk

In [ ]:
#with open("b64DecodedImage1.png", "wb") as fh:
#    fh.write(decoded_image)

We will be keeping this image in memory since we will be making further modifications before saving the final image.

In [ ]:
byte_stream = BytesIO(decoded_image)

Let's view the returned iamge from Imagen

In [ ]:
genImage = Image.open(byte_stream).convert("RGBA")
genImage.convert("RGB")

You can save the final image by uncommenting the line below.

In [ ]:
genImage.save("result.png")

Congratulations, you just created your first outpainted image using Imagen!